In [1]:
import pandas as pd
import numpy as np
import time
from time import gmtime, strftime

In [2]:
%%time
s3_athena_file = 's3://lb.athena.queries.result/products_reviews_queries/2021/06/15/07d38fc4-1a3e-401f-9832-7b7529389717.csv'
data = pd.read_csv(s3_athena_file, error_bad_lines=False, dtype='str')

CPU times: user 1.54 s, sys: 238 ms, total: 1.78 s
Wall time: 2.94 s


In [3]:
data.dropna(inplace=True)

print(data.shape)
print(data.columns)

data.head()

(145427, 16)
Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'star_rating', 'helpful_votes',
       'total_votes', 'vine', 'verified_purchase', 'review_headline',
       'review_body', 'review_date', 'year', 'product_category'],
      dtype='object')


,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,891276,R3LGIYKBEHXTYB,B004RMK57U,53534661,Playstation Plus Subscription,5,0,0,N,Y,Fast purchase,"It was a good deal, worked perfectly ty amazon",16391,2014,Digital_Video_Games
1,US,50451909,R14QEUQI9OR0YV,B00GXHISJE,599419294,Fallout 3: Game of The Year Edition,2,0,1,N,Y,Freezes!,I have Win 7 and I'm running with 6 gigs of RA...,16170,2014,Digital_Video_Games
2,US,34443239,R4OM7NI4IM1Y2,B00K59HKIQ,384246568,Playstation Network Card,5,0,0,N,Y,Super fast !,Super fast !,16391,2014,Digital_Video_Games
3,US,4461282,R10DI66H4ZRF9Q,B0087STJLS,296282987,Battlefield 3: Premium Season Pass,5,0,0,N,Y,"A lot of really cool maps, enjoy guarantee.","the maps are great , are full of details this ...",16170,2014,Digital_Video_Games
4,US,16183275,R2X6KKO5JTNP27,B00JLK6ULS,216162264,Sid Meier's Civilization: Beyond Earth,2,0,1,N,Y,Very Disappointed.,I was really excited for this game and picked ...,16391,2014,Digital_Video_Games


In [4]:
%%time
data['review_body'] = data['review_headline'] + ' ' + data['review_body']

CPU times: user 76.3 ms, sys: 24.9 ms, total: 101 ms
Wall time: 245 ms


In [5]:
%%time
data = data[['review_id', 'product_id','star_rating','review_body']]

CPU times: user 79.2 ms, sys: 21.9 ms, total: 101 ms
Wall time: 122 ms


In [7]:
data.head()

,review_id,product_id,star_rating,review_body
0,R3QK7ITZLJTWLF,B002BU0DPE,1,Cheaply made. The headset is shoddy. The wires...
1,R2FNT507ZCB4GE,B001GM6C26,3,not bad being a fan of hidden pictures this ga...
2,R1LVPFY1GV1NLA,B00178630A,1,"Addictive, but not fun Let me give you a quick..."
3,R3MGDCF0BFTYQM,B0012NAXE2,2,THE POOREST MADDEN GAME EVER When I first got ...
4,RZD8INNNJ46OZ,B0038TR60M,2,Good case but still disappointed The case is e...


In [8]:
data.star_rating.unique()

array(['1', '3', '2', '4', '5'], dtype=object)

In [9]:
%%time
data['label'] = data.star_rating.map({
    '1':'__label__negative__',
    '2':'__label__negative__',
    '3':'__label__neutral__',
    '4':'__label__positive__',
    '5':'__label__positive__'
})

CPU times: user 174 ms, sys: 3.64 ms, total: 177 ms
Wall time: 224 ms


In [10]:
data.head()

,review_id,product_id,star_rating,review_body,label
0,R3QK7ITZLJTWLF,B002BU0DPE,1,Cheaply made. The headset is shoddy. The wires...,__label__negative__
1,R2FNT507ZCB4GE,B001GM6C26,3,not bad being a fan of hidden pictures this ga...,__label__neutral__
2,R1LVPFY1GV1NLA,B00178630A,1,"Addictive, but not fun Let me give you a quick...",__label__negative__
3,R3MGDCF0BFTYQM,B0012NAXE2,2,THE POOREST MADDEN GAME EVER When I first got ...,__label__negative__
4,RZD8INNNJ46OZ,B0038TR60M,2,Good case but still disappointed The case is e...,__label__negative__


## Tokenize reviews

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
%%time
data['review_body'] = data['review_body'].apply(nltk.word_tokenize)

In [ ]:
%%time
data['review_body'] = data.apply(lambda row: " ".join(row['review_body']).lower(), axis=1)